# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
CREATE TABLE employees (
  ID_usr INT,  -- User ID
  name VARCHAR(50) -- Name of the employee
);

-- Sample rows
-- (1, 'Alice')
-- (2, 'Bob')
-- (3, 'Charlie')

CREATE TABLE salary (
  ID_usr INT, -- User ID
  year DATE, -- Year of salary data
  salary FLOAT -- Salary amount for the given year
);

-- Sample rows
-- (1, '2021-01-01', 55000.00)
-- (2, '2021-01-01', 45000.00)
-- (3, '2021-01-01', 50000.00)

CREATE TABLE studies (
  ID INT, -- Unique ID for the record
  ID_usr INT, -- User ID
  educational_level INT, -- Level of education (1 = Bachelor, 2 = Master, 3 = PhD)
  Institution VARCHAR(100), -- Name of the institution
  Years DATE, -- Date of entry into studies
  Speciality VARCHAR(50) -- Specialty of the studies
);

-- Sample rows
-- (1, 1, 1, 'University A', '2020-09-01', 'Computer Science')
-- (2, 2, 2, 'University B', '2019-09-01', 'Electrical Engineering')
-- (3, 3, 3, 'University C', '2018-09-01', 'Physics')

""" } ]


In [11]:
# FEW SHOT SAMPLES
context.append({'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

-- Example Query 1: Get the names of employees who have a salary greater than 50,000.
-- SQL:
-- SELECT name FROM employees 
-- JOIN salary ON employees.ID_usr = salary.ID_usr
-- WHERE salary.salary > 50000;

-- Example Query 2: Find the average salary for each educational level.
-- SQL:
-- SELECT educational_level, AVG(salary.salary) AS average_salary
-- FROM studies 
-- JOIN salary ON studies.ID_usr = salary.ID_usr
-- GROUP BY educational_level;

-- Example Query 3: Get the names of employees who graduated with a Master's degree.
-- SQL:
-- SELECT name 
-- FROM employees 
-- JOIN studies ON employees.ID_usr = studies.ID_usr
-- WHERE studies.educational_level = 2;
"""})

In [4]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [5]:
#new
context_user = context.copy()
print(return_CCRMSQL("""What is the total salary paid to employees in the year 2021?""", context_user))

To calculate the total salary paid to employees in the year 2021, you can use the following SQL query:

```sql
SELECT SUM(salary) AS total_salary_2021
FROM salary
WHERE year = '2021-01-01';
```

This query will sum up all the salaries from the `salary` table where the year is '2021-01-01' and provide the total salary paid to employees in the year 2021.


In [6]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List all employees who graduated with a Master's degree from 'University B'.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 'Master' AND s.Institution = 'University B';
```

This SQL query retrieves the names of employees who graduated with a Master's degree from 'University B' by joining the employees and studies tables on the ID_usr column and filtering the results based on the educational level and institution.


In [7]:
#new
print(return_CCRMSQL("Which employee has the highest salary, and what is the amount?", context_user))

To find the employee with the highest salary and the corresponding amount, you can use the following SQL query:

```sql
SELECT e.name AS Employee_Name, s.salary AS Salary_Amount
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary = (SELECT MAX(salary) FROM salary);
```

This query will return the name of the employee with the highest salary and the amount of that salary.


In [8]:
#old
print(return_CCRMSQL("Get the names of employees who are enrolled in Computer Science studies.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 'Computer Science';
```

This SQL query retrieves the names of employees who are enrolled in Computer Science studies by joining the "employees" table with the "studies" table on the ID_usr field and filtering the results based on the educational level being "Computer Science".


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?